In [1]:
import google.datalab.bigquery as bq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# apm_db_os 20180129 28일 15~19시

In [2]:
%%bq query --name apm_D28_15to19
SELECT time, os_id, avg_value
FROM(
  SELECT *, UNIX_SECONDS(time) as unixtime
  FROM `xmdw31.ods_mga.apm_db_os_stat_temp_20180129` AS os_29
)
WHERE unixtime>1517151600 and unixtime<=1517166000 and (os_id = 0 or os_id = 1 or os_id = 2 or os_id = 9)
ORDER BY time, os_id

In [3]:
%%bq sample --query apm_D28_15to19 --count 500

time,os_id,avg_value
2018-01-28 15:00:01,0,80.0
2018-01-28 15:00:01,1,100.0
2018-01-28 15:00:01,2,30.0
2018-01-28 15:00:01,9,13325220.0
2018-01-28 15:00:02,0,3930.0
2018-01-28 15:00:02,1,1640.0
2018-01-28 15:00:02,2,490.0
2018-01-28 15:00:02,9,13097124.0
2018-01-28 15:00:03,0,610.0
2018-01-28 15:00:03,1,310.0


In [4]:
apm_D28_15to19_sql = 'SELECT UNIX_SECONDS(time) as unixtime, os_id, avg_value FROM( SELECT *, UNIX_SECONDS(time) as unixtime FROM `xmdw31.ods_mga.apm_db_os_stat_temp_20180129` AS os_29 ) WHERE unixtime>1517151600 and unixtime<=1517166000 and (os_id = 0 or os_id = 1 or os_id = 2 or os_id = 9) ORDER BY time, os_id'

apm_D28_15to19_query = bq.Query(apm_D28_15to19_sql)

apm_D28_15to19_df = apm_D28_15to19_query.execute(output_options = bq.QueryOutput.dataframe()).result()

In [5]:
print(apm_D28_15to19_df.head(10))

     unixtime  os_id   avg_value
0  1517151601      0        80.0
1  1517151601      1       100.0
2  1517151601      2        30.0
3  1517151601      9  13325220.0
4  1517151602      0      3930.0
5  1517151602      1      1640.0
6  1517151602      2       490.0
7  1517151602      9  13097124.0
8  1517151603      0       610.0
9  1517151603      1       310.0


# 한 시간을 하나의 row로 flatten 시키자

In [6]:
apm_D28_15to19_arr = apm_D28_15to19_df.as_matrix()
print(apm_D28_15to19_arr)
print(apm_D28_15to19_arr.shape)

[[1.5171516e+09 0.0000000e+00 8.0000000e+01]
 [1.5171516e+09 1.0000000e+00 1.0000000e+02]
 [1.5171516e+09 2.0000000e+00 3.0000000e+01]
 ...
 [1.5171660e+09 1.0000000e+00 0.0000000e+00]
 [1.5171660e+09 2.0000000e+00 0.0000000e+00]
 [1.5171660e+09 9.0000000e+00 1.3405900e+07]]
(57600, 3)


In [9]:
result_arr = [[]]

for i in range(14400):
  temp_arr = []
  for j in range(4):
    if(j==0):
      temp_arr = [np.append(temp_arr, i)]
    temp_arr = [np.append(temp_arr, apm_D28_15to19_arr[4*i+j][2])]
    if(j == 3):
      if(i == 0):
        result_arr = [np.append(result_arr, temp_arr)]
      else:
        result_arr = np.append(result_arr, temp_arr, axis = 0)


In [10]:
print(result_arr)
print(result_arr.shape)

[[0.0000000e+00 8.0000000e+01 1.0000000e+02 3.0000000e+01 1.3325220e+07]
 [1.0000000e+00 3.9300000e+03 1.6400000e+03 4.9000000e+02 1.3097124e+07]
 [2.0000000e+00 6.1000000e+02 3.1000000e+02 1.3000000e+02 1.3081448e+07]
 ...
 [1.4397000e+04 1.2900000e+03 4.6000000e+02 4.3000000e+02 1.3405900e+07]
 [1.4398000e+04 3.8000000e+02 1.3000000e+02 1.0000000e+02 1.3405900e+07]
 [1.4399000e+04 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.3405900e+07]]
(14400, 5)


# Big Query 로 보내자

In [11]:
output_df = pd.DataFrame(result_arr)

output_df.columns = ['index', 'os_value_0', 'os_value_1', 'os_value_2', 'os_value_9']

In [12]:
output_df.to_gbq('z_intern.apm_os_D18_H_15to19_os_4s', 'xmdw31', if_exists = 'replace', chunksize = 2000)




Load is 13% Complete
Load is 27% Complete
Load is 41% Complete
Load is 55% Complete
Load is 69% Complete
Load is 83% Complete
Load is 97% Complete
Load is 100% Complete




# ora 테이블과 join..!

In [13]:
%%bq query 
--name ora_data1
SELECT *
FROM `xmdw31.z_intern.ora_D18_H_15to19_stat_1to13` AS wp_29

stat_value_1,stat_value_2,stat_value_3,stat_value_4,stat_value_5,stat_value_6,stat_value_7,stat_value_8,stat_value_9,stat_value_10,stat_value_11,stat_value_12,stat_value_13
732567590.0,224897.0,26.0,41209257.0,69.0,34489719.0,3622857.0,733397674.0,181992500.0,65910291.0,18598.0,37499.0,37496.0
732567489.0,224897.0,26.0,41209253.0,69.0,34489713.0,3622857.0,733397574.0,181992489.0,65910291.0,18599.0,37499.0,37496.0
732565866.0,224894.0,26.0,41209108.0,69.0,34489625.0,3622843.0,733395942.0,181991458.0,65909421.0,18598.0,37499.0,37496.0
732565866.0,224894.0,26.0,41209108.0,69.0,34489625.0,3622843.0,733395942.0,181991462.0,65909421.0,18598.0,37499.0,37496.0
732565762.0,224894.0,26.0,41209105.0,69.0,34489621.0,3622841.0,733395838.0,181991442.0,65909296.0,18598.0,37499.0,37496.0
732565760.0,224894.0,26.0,41209105.0,69.0,34489621.0,3622840.0,733395836.0,181991433.0,65909296.0,18598.0,37499.0,37496.0
732565760.0,224894.0,26.0,41209105.0,69.0,34489621.0,3622840.0,733395836.0,181991437.0,65909296.0,18598.0,37499.0,37496.0
732565829.0,224894.0,26.0,41209106.0,68.0,34489623.0,3622842.0,733395905.0,181991447.0,65909361.0,18598.0,37499.0,37496.0
732565276.0,224894.0,26.0,41209095.0,69.0,34489601.0,3622837.0,733395352.0,181991296.0,65909172.0,18598.0,37499.0,37496.0
732565276.0,224894.0,26.0,41209095.0,69.0,34489601.0,3622837.0,733395352.0,181991307.0,65909172.0,18598.0,37499.0,37496.0


In [14]:
ora_D28_15to19_sql = 'SELECT * FROM `xmdw31.z_intern.ora_D18_H_15to19_stat_1to13` AS wp_29'
ora_D28_15to19_query = bq.Query(ora_D28_15to19_sql)

ora_D28_15to19_df = ora_D28_15to19_query.execute(output_options = bq.QueryOutput.dataframe()).result()

In [15]:
print(ora_D28_15to19_df.head(20))

    stat_value_1  stat_value_2  stat_value_3  stat_value_4  stat_value_5  \
0    732567590.0      224897.0          26.0    41209257.0          69.0   
1    732567489.0      224897.0          26.0    41209253.0          69.0   
2    732565866.0      224894.0          26.0    41209108.0          69.0   
3    732565866.0      224894.0          26.0    41209108.0          69.0   
4    732565762.0      224894.0          26.0    41209105.0          69.0   
5    732565760.0      224894.0          26.0    41209105.0          69.0   
6    732565760.0      224894.0          26.0    41209105.0          69.0   
7    732565829.0      224894.0          26.0    41209106.0          68.0   
8    732565276.0      224894.0          26.0    41209095.0          69.0   
9    732565276.0      224894.0          26.0    41209095.0          69.0   
10   732565276.0      224894.0          26.0    41209095.0          69.0   
11   732565276.0      224894.0          26.0    41209095.0          69.0   
12   7325657

In [ ]:
join 하기